# Final Project

By: Rashaad Ratliff-Brown and Abhijith Tammanagari

In [1]:
import pandas as pd
import os
import re
#import spacy
import gensim
from gensim.models.phrases import Phrases, Phraser
from time import time 
import multiprocessing
from gensim.models import Word2Vec
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale
import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import confusion_matrix
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
ks = pd.read_csv("kickstarter-data-modified.csv")
ks.head()

,Blurbs,Backed
0,"'Catalysts, Explorers & Secret Keepers: Women ...",1
1,A unique handmade picture book for kids & art ...,0
2,A horror comedy about a repairman who was in t...,1
3,The Johnny Wander autobio omnibus you've all b...,1
4,The vision for this project is the establishme...,0


In [3]:
df_clean = ks

t = time()


tokenizer = RegexpTokenizer(r'\w+')
df_clean['clean'] = df_clean['Blurbs'].astype('str') 
df_clean.dtypes

df_clean["tokens"] = df_clean["clean"].apply(tokenizer.tokenize)
# delete Stop Words

print('Time to tokenize everything: {} mins'.format(round((time() - t) / 60, 2)))
df_clean.head()

Time to tokenize everything: 0.0 mins


,Blurbs,Backed,clean,tokens
0,"'Catalysts, Explorers & Secret Keepers: Women ...",1,"'Catalysts, Explorers & Secret Keepers: Women ...","[Catalysts, Explorers, Secret, Keepers, Women,..."
1,A unique handmade picture book for kids & art ...,0,A unique handmade picture book for kids & art ...,"[A, unique, handmade, picture, book, for, kids..."
2,A horror comedy about a repairman who was in t...,1,A horror comedy about a repairman who was in t...,"[A, horror, comedy, about, a, repairman, who, ..."
3,The Johnny Wander autobio omnibus you've all b...,1,The Johnny Wander autobio omnibus you've all b...,"[The, Johnny, Wander, autobio, omnibus, you, v..."
4,The vision for this project is the establishme...,0,The vision for this project is the establishme...,"[The, vision, for, this, project, is, the, est..."


In [4]:
backedData = df_clean[df_clean['Backed'] == 1]
unbackedData = df_clean[df_clean['Backed'] == 0]
backedTokens = backedData['tokens'].tolist()
backedWords = []
for token in backedTokens:
    backedWords += token
unbackedTokens = unbackedData['tokens'].tolist()
unbackedWords = []
for token in unbackedTokens:
    unbackedWords += token

In [5]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [6]:
#WORD2VEC()
#removed size
cores = multiprocessing.cpu_count() # Count the number of cores in a computer, important for a parameter of the model
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

#BUILD_VOCAB()
t = time()
w2v_model.build_vocab(df_clean["tokens"], progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

#TRAIN()
w2v_model.train(df_clean["tokens"], total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.0 mins
Time to train the model: 0.04 mins


In [8]:
#First defining the X (input), and the y (output)
y = df_clean['Backed'].values
X = np.array(df_clean["tokens"])

#And here is the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [17]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [9]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

vocab size : 800


In [27]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        #print(word)
        try:
            vec += w2v_model.wv[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [22]:
w2v_model.wv.vector_size
w2v_model['The']

TypeError: 'Word2Vec' object is not subscriptable

In [19]:
w2v_model.wv.most_similar(positive=["good"])
print(w2v_model)

Word2Vec(vocab=471, vector_size=100, alpha=0.03)


In [28]:
train_vecs_w2v = np.concatenate([buildWordVector(z, 100) for z in map(lambda x: x, X_train)])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 100) for z in map(lambda x: x, X_test)])
test_vecs_w2v = scale(test_vecs_w2v)

print ('shape for training set : ',train_vecs_w2v.shape,
      '\nshape for test set : ', test_vecs_w2v.shape)

shape for training set :  (3200, 100) 
shape for test set :  (800, 100)


In [31]:
model = Sequential()

model.add(Dense(128, activation='relu', input_dim=100))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               12928     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 13,057
Trainable params: 13,057
Non-trainable params: 0
_________________________________________________________________


In [32]:
history = model.fit(train_vecs_w2v, y_train, epochs=20, batch_size=50,
                   validation_data=(test_vecs_w2v,y_test))
loss, accuracy = model.evaluate(train_vecs_w2v, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_vecs_w2v, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/20
64/64 [==============================] - 1s 4ms/step - loss: 0.7782 - accuracy: 0.6125 - val_loss: 0.6260 - val_accuracy: 0.7100
Epoch 2/20
64/64 [==============================] - 0s 2ms/step - loss: 0.8030 - accuracy: 0.6328 - val_loss: 0.6242 - val_accuracy: 0.7100
Epoch 3/20
64/64 [==============================] - 0s 2ms/step - loss: 0.7699 - accuracy: 0.6153 - val_loss: 0.6227 - val_accuracy: 0.7150
Epoch 4/20
64/64 [==============================] - 0s 2ms/step - loss: 0.7902 - accuracy: 0.6303 - val_loss: 0.6214 - val_accuracy: 0.7175
Epoch 5/20
64/64 [==============================] - 0s 2ms/step - loss: 0.7939 - accuracy: 0.6134 - val_loss: 0.6199 - val_accuracy: 0.7225
Epoch 6/20
64/64 [==============================] - 0s 3ms/step - loss: 0.8093 - accuracy: 0.6250 - val_loss: 0.6186 - val_accuracy: 0.7225
Epoch 7/20
64/64 [==============================] - 0s 3ms/step - loss: 0.7697 - accuracy: 0.6356 - val_loss: 0.6174 - val_accuracy: 0.7225
Epoch 8/20
64/64 [==

In [ ]:
def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
plot_history(history)

#### Naive Bayes

In [46]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
ks = pd.read_csv("kickstarter-data-modified.csv")
ks.head()
model = make_pipeline(TfidfVectorizer(), MultinomialNB())
ksBlurbs = ks['Blurbs'].tolist()
ksBacked = ks['Backed'].tolist()
X_train, X_test, y_train, y_test = train_test_split(ksBlurbs, ksBacked, test_size = 0.2, random_state = 0)
model.fit(X_train, y_train)
labels = model.predict(X_test)
correct = 0
for x in range(0,len(labels)):
    if labels[x] == y_test[x]:
        correct += 1
accuracy = correct / len(labels)
print(accuracy)

0.815


Step 1: Choose an NLP problem: Classification (Predict success on Kickstarter based on Kickstarter summary) 

Step 2: Identify or construct a solution based on a generative probabilistic (language) model. Describe the model in detail and develop a solution using parameter inference (and/or decoding).


- split data 25/75
- Gaussian Naive Bayes is implemented 
- training data is fit to model 
- predictions
- confusion matrix 

Step 3: Identify or construct a solution based on a (discriminative) neural network. Describe the network structure in detail and develop a solution using gradient descent.

For our neural network approach, first we take the cleaned data and then using googles news word to vector model we can convert our Kickstarter blurbs into vectors as well split into test and training. Then we feed that into our sequential neural network with the following layers a relu activation function followed by a dropout layer of 0.7 and then a sigmoid layer with a binary cross entropy loss function.

Step 4: Apply both approaches to synthetic data that you generate according to the generative model from Step 2. Evaluate the results qualitatively and quantitatively. Highlight situations where each approach performs well and poorly.

Step 5: Apply both approaches to “real” data acquired legally. Evaluate the results qualitatively and quantitatively. Highlight situations where each approach performs well and poorly. Any unusual/unexpected results require explanation (and frankly, probably debugging).

Step 6: Discuss pros and cons of the two approaches